In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install uproot

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

In [ ]:
!pip install uproot tqdm numexpr google-colab

In [ ]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import pickle
import torch
from torch_geometric.data import Data
import torch_geometric
import tqdm
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.loader import DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
particles_file = uproot.open("/content/drive/MyDrive/IRIS-HEP_DFEI/particles_data.root")

In [ ]:
particles_tree = particles_file['Particles']

In [ ]:
relevant_particles_keys = ['ExpandedEventNumber',
                          'ExpandedParticleIndex',
                          'ExpandedPrimaryHeavyHadronIndex',
                          'FromPrimaryHeavyHadron',
                          'ParticleType',
                          'associated_ip_reco',
                          'associated_xPV_reco',
                          'associated_yPV_reco',
                          'associated_zPV_reco',
                          'charge',
                          'inGeomAcc',
                          'isCharged',
                          'pt_reco',
                          'px_reco',
                          'py_reco',
                          'pz_reco',
                          'xProd_reco',
                          'yProd_reco',
                          'zProd_reco']

In [ ]:
df_particles = particles_tree.arrays(relevant_particles_keys, library='pd')

In [ ]:
df_particles

,ExpandedEventNumber,ExpandedParticleIndex,ExpandedPrimaryHeavyHadronIndex,FromPrimaryHeavyHadron,ParticleType,associated_ip_reco,associated_xPV_reco,associated_yPV_reco,associated_zPV_reco,charge,inGeomAcc,isCharged,pt_reco,px_reco,py_reco,pz_reco,xProd_reco,yProd_reco,zProd_reco
0,0.0,1057.0,-1.0,0.0,2.0,-1.000000,-1.000000,-1.000000,-1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1106.0,-1.0,0.0,0.0,0.015194,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.286947,-0.153205,-0.242625,2.750225,-0.642510,-0.880182,-19.230768
2,0.0,1110.0,-1.0,0.0,0.0,0.006571,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.576406,-0.555643,-0.153313,13.076964,-0.511701,-0.120768,-19.230768
3,0.0,1111.0,-1.0,0.0,0.0,0.010698,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.922732,-0.865247,0.320597,6.068925,-1.524306,0.530222,-19.230768
4,0.0,1112.0,-1.0,0.0,0.0,0.010318,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.253314,-0.246892,0.056676,1.878958,-1.392859,0.292333,-19.230768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374328,6161.0,2430.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.657934,-0.611031,0.243963,3.661947,0.000000,0.000000,0.000000
1374329,6161.0,2431.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.559823,-0.523263,-0.198992,2.484762,0.000000,0.000000,0.000000
1374330,6161.0,2444.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.313306,-0.313267,0.004922,1.603106,0.000000,0.000000,0.000000
1374331,6161.0,2445.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.416930,-0.082793,0.408627,2.115970,0.000000,0.000000,0.000000


In [ ]:
df_Y = df_particles.loc[:, ['ExpandedEventNumber', 'ExpandedParticleIndex', 'FromPrimaryHeavyHadron']]

In [ ]:
df_Y

,ExpandedEventNumber,ExpandedParticleIndex,FromPrimaryHeavyHadron
0,0.0,1057.0,0.0
1,0.0,1106.0,0.0
2,0.0,1110.0,0.0
3,0.0,1111.0,0.0
4,0.0,1112.0,0.0
...,...,...,...
1374328,6161.0,2430.0,0.0
1374329,6161.0,2431.0,0.0
1374330,6161.0,2444.0,0.0
1374331,6161.0,2445.0,0.0


In [ ]:
df_particles = df_particles.set_index('ExpandedParticleIndex')

In [ ]:
df_particles

,ExpandedEventNumber,ExpandedPrimaryHeavyHadronIndex,FromPrimaryHeavyHadron,ParticleType,associated_ip_reco,associated_xPV_reco,associated_yPV_reco,associated_zPV_reco,charge,inGeomAcc,isCharged,pt_reco,px_reco,py_reco,pz_reco,xProd_reco,yProd_reco,zProd_reco
ExpandedParticleIndex,,,,,,,,,,,,,,,,,,
1057.0,0.0,-1.0,0.0,2.0,-1.000000,-1.000000,-1.000000,-1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1106.0,0.0,-1.0,0.0,0.0,0.015194,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.286947,-0.153205,-0.242625,2.750225,-0.642510,-0.880182,-19.230768
1110.0,0.0,-1.0,0.0,0.0,0.006571,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.576406,-0.555643,-0.153313,13.076964,-0.511701,-0.120768,-19.230768
1111.0,0.0,-1.0,0.0,0.0,0.010698,-0.078272,-0.006414,-29.304338,-1.0,1.0,1.0,0.922732,-0.865247,0.320597,6.068925,-1.524306,0.530222,-19.230768
1112.0,0.0,-1.0,0.0,0.0,0.010318,-0.078272,-0.006414,-29.304338,1.0,1.0,1.0,0.253314,-0.246892,0.056676,1.878958,-1.392859,0.292333,-19.230768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2430.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.657934,-0.611031,0.243963,3.661947,0.000000,0.000000,0.000000
2431.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.559823,-0.523263,-0.198992,2.484762,0.000000,0.000000,0.000000
2444.0,6161.0,-1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.313306,-0.313267,0.004922,1.603106,0.000000,0.000000,0.000000


In [ ]:
num_of_events = 6161

In [ ]:
relevant_edge_features = [
                          'DOCA_reco',
                          'FromSameAssociatedPV_reco',
                          'delta_z0_reco',
                          'p1_isCharged',
                          'p2_isCharged',
                          'theta_reco',
                          'trdist_reco'
                          ]

In [ ]:
K = 5

In [ ]:
list_of_graphs = []

In [ ]:
for event_number in tqdm.tqdm(range(num_of_events)):
  
  df_p = df_particles.loc[df_particles.ExpandedEventNumber == event_number]
  mapping = {index: i for i, index in enumerate(df_p.index)}
  x = df_p.loc[:, ~df_p.columns.isin(['ExpandedParticleIndex', 'ExpandedEventNumber', 'FromPrimaryHeavyHadron', 'ExpandedPrimaryHeavyHadronIndex', 'ParticleType', 'inGeomAcc'])]
  x = torch.tensor(x.values, dtype=torch.float)

  filename = f"event_{event_number}_top_{K}_relations_filtered_by_DOCA_for_each_particle.csv"
  path = f'path_to_folder_with_event_relations/{filename}'
  df_r = pd.read_csv(path)
  edges = []
  edge_attr = []
  for _, row in df_r.iterrows():
    sender = mapping[row['FirstParticleIndex']]
    reciever = mapping[row['SecondParticleIndex']]
    edge_1 = [sender, reciever]
    edge_2 = [reciever, sender]
    edges.append(edge_1)
    edges.append(edge_2)
    edge_features = list(row[relevant_edge_features].values)
    edge_attr.append(edge_features)
    edge_attr.append(edge_features)
  edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)


  y = torch.tensor(df_Y.loc[df_Y.ExpandedEventNumber == event_number]['FromPrimaryHeavyHadron'].values)
  y = y.reshape(-1, 1)

  graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

  list_of_graphs.append(graph)

100%|██████████| 6161/6161 [1:36:51<00:00,  1.06it/s]


In [ ]:
import pickle 

with open('/content/drive/MyDrive/IRIS-HEP_DFEI/list_of_graphs_from_filtered_data_k5.pickle', 'wb') as handle:
    pickle.dump(list_of_graphs, handle, protocol=pickle.HIGHEST_PROTOCOL)